In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

import matplotlib.pyplot as plt
import seaborn as sns
import os

In [13]:
df_complete = pd.read_csv('../data/data_complete.csv', index_col=0)
df_mean = pd.read_csv('../data/data_mean.csv', index_col=0)
df_ml = pd.read_csv('../data/data_ml.csv', index_col=0)
df_mice = pd.read_csv('../data/data_mice.csv', index_col=0)

/var/folders/0r/r2dplk4d72d437ygr7ycq8kw0000gn/T/ipykernel_20220/3215121032.py:4: DtypeWarning: Columns (47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mice = pd.read_csv('../data/data_mice.csv', index_col=0)


In [14]:
# delete Csection.Urgency for predicting delivery method
del df_complete['Csection.Urgency']
del df_mean['Csection.Urgency']
del df_ml['Csection.Urgency']
del df_mice['Csection.Urgency'] 

In [15]:
method_complete = df_complete.pop('Method.of.Delivery.A')
method_mean = df_mean.pop('Method.of.Delivery.A')
method_ml = df_ml.pop('Method.of.Delivery.A')
method_mice = df_mice.pop('Method.of.Delivery.A')

In [16]:
labels_complete = pd.get_dummies(method_complete)['C/S']
labels_mean = pd.get_dummies(method_mean)['C/S']
labels_ml = pd.get_dummies(method_ml)['C/S']
labels_mice = pd.get_dummies(method_mice)['C/S']

In [17]:
print(method_complete.value_counts())

Vag    3052
C/S      87
Name: Method.of.Delivery.A, dtype: int64


In [18]:
print(labels_complete.value_counts())

0    3052
1      87
Name: C/S, dtype: int64


###  Model (use. LASSO to see which dataset has the best performance)

In [19]:
from functions import helpers
from functions import models
from functions import classifiers

2024-07-05 03:50:43.871587: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
features_complete = pd.get_dummies(df_complete)
features_mean = pd.get_dummies(df_mean)
features_ml = pd.get_dummies(df_ml)
features_mice = pd.get_dummies(df_mice)

### data : complete

In [21]:
X_train, X_test, y_train, y_test = helpers.split_and_scale_data(features_complete, labels_complete)

In [22]:
y_train.value_counts()

0    2139
1      58
Name: C/S, dtype: int64

In [23]:
y_test.value_counts()

0    913
1     29
Name: C/S, dtype: int64

In [24]:
X_train, y_train = helpers.perform_SMOTE(X_train, y_train)
y_train.value_counts()

0    2139
1    2139
Name: C/S, dtype: int64

In [25]:
LASSO_train, LASSO_test, output, model = models.run_LASSO2(X_train, X_test, y_train, y_test, study='DeliveryMethod')



/Users/menglinghe/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [26]:
y_pred = model.predict(X_test)
y_pred_binary = [0 if elem <= 0.5 else 1 for elem in y_pred]

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred_binary))
print('Precision:', precision_score(y_test, y_pred_binary))
print('Recall:', recall_score(y_test, y_pred_binary))

AUC: 1.0
Accuracy: 1.0
Precision: 1.0
Recall: 1.0


In [28]:
pd.set_option('display.max_rows', 350)
output[(output['significant'] == True) & (output['coef'] < 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
111,DeliveryMethod,Episiotomy_RT_Medio,-1.358688,0.114967,-1.440931,-1.276445,10,True,0.710375
39,DeliveryMethod,County_meigs,-0.548813,0.306395,-0.767995,-0.329632,9,True,0.000000
66,DeliveryMethod,Amniotic.Fluid.Color_heavy meconium,-0.795064,0.553252,-1.190836,-0.399291,7,True,0.000000
130,DeliveryMethod,laceration_type_cervical,-0.801933,0.431088,-1.110315,-0.493550,8,True,0.000000
131,DeliveryMethod,laceration_type_perineal,-0.442698,0.382515,-0.716333,-0.169064,6,True,0.000000
155,DeliveryMethod,Forceps.a_Mid,-0.433750,0.234554,-0.601539,-0.265960,8,True,0.000000
166,DeliveryMethod,Delivery.Month_Mar,-0.024450,0.031943,-0.047300,-0.001599,6,True,0.000000
193,DeliveryMethod,Premature.Rom_No,-0.106111,0.121896,-0.193310,-0.018912,8,True,0.000000


In [29]:
output[(output['significant'] == True) & (output['coef'] > 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
112,DeliveryMethod,Oxytocin_Ind,1.247836,0.117002,1.164138,1.331535,10,True,0.461447
113,DeliveryMethod,Weight.Gain.Pounds,0.216221,0.025624,0.197891,0.234551,10,True,0.001542
3,DeliveryMethod,County_cocke,0.164850,0.135099,0.068207,0.261494,9,True,0.000000
5,DeliveryMethod,County_scott,0.465821,0.181017,0.336329,0.595312,10,True,0.000000
13,DeliveryMethod,Premature.Rom_Yes,0.263271,0.168024,0.143074,0.383468,8,True,0.000000
38,DeliveryMethod,Provider.On.Admission_UWS/SUD,0.030146,0.037528,0.003301,0.056992,5,True,0.000000
41,DeliveryMethod,Adm.Cigarettes_former,0.183932,0.036884,0.157547,0.210318,10,True,0.000000
67,DeliveryMethod,Born.In.Route.A_No,0.280054,0.031346,0.257630,0.302477,10,True,0.000000
76,DeliveryMethod,County_blount,0.089015,0.072931,0.036843,0.141187,7,True,0.000000
94,DeliveryMethod,delivery_anesthesia_epidural,0.353785,0.030572,0.331915,0.375654,10,True,0.000000


### data : mean

In [30]:
X_train, X_test, y_train, y_test = helpers.split_and_scale_data(features_mean, labels_mean)

In [31]:
y_train.value_counts()

0    8631
1    3703
Name: C/S, dtype: int64

In [32]:
y_test.value_counts()

0    3691
1    1595
Name: C/S, dtype: int64

In [33]:
X_train, y_train = helpers.perform_SMOTE(X_train, y_train)
y_train.value_counts()

1    8631
0    8631
Name: C/S, dtype: int64

In [34]:
LASSO_train, LASSO_test, output, model = models.run_LASSO2(X_train, X_test, y_train, y_test, study='DeliveryMethod')

/Users/menglinghe/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [35]:
y_pred = model.predict(X_test)
y_pred_binary = [0 if elem <= 0.5 else 1 for elem in y_pred]

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred_binary))
print('Precision:', precision_score(y_test, y_pred_binary))
print('Recall:', recall_score(y_test, y_pred_binary))

AUC: 0.9953708461401918
Accuracy: 0.9943246311010215
Precision: 0.9893683552220137
Recall: 0.9918495297805643


In [36]:
pd.set_option('display.max_rows', 350)
output[(output['significant'] == True) & (output['coef'] < 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
133,DeliveryMethod,Vacuum.Extraction.A_Failed,-3.477424,0.904275,-4.124304,-2.830545,10,True,0.365545
158,DeliveryMethod,Provider.On.Admission_ROC/WSC,-0.361948,0.199309,-0.504526,-0.219371,9,True,0.001401
46,DeliveryMethod,Gestational.Status.A_2,-0.447440,0.237588,-0.617401,-0.277480,10,True,0.001293
22,DeliveryMethod,Oxytocin_Aug,-0.180981,0.059147,-0.223292,-0.138669,10,True,0.000811
2,DeliveryMethod,Preterm.Labor_No,-0.142920,0.138192,-0.241776,-0.044063,10,True,0.000793
39,DeliveryMethod,Provider.On.Admission_ROCK,-0.227381,0.128861,-0.319563,-0.135199,9,True,0.000515
19,DeliveryMethod,infant_complications_A_acidosis,-0.137347,0.073437,-0.189881,-0.084814,9,True,0.000494
35,DeliveryMethod,delivery_anesthesia_spinal,-0.510680,0.177322,-0.637528,-0.383831,10,True,0.000467
68,DeliveryMethod,County_sullivan,-0.280251,0.139959,-0.380371,-0.180130,9,True,0.000452
163,DeliveryMethod,County_onslow,-0.176492,0.130443,-0.269805,-0.083178,8,True,0.000410


In [37]:
output[(output['significant'] == True) & (output['coef'] > 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
134,DeliveryMethod,VBAC.A_N_A,1.490031,0.838984,0.889858,2.090204,9,True,0.032905
135,DeliveryMethod,Csection.Incidence_N_A,1.102374,0.729368,0.580615,1.624132,9,True,0.008620
17,DeliveryMethod,Adm.Alcohol_YES,1.207705,0.251517,1.027780,1.387629,10,True,0.008063
119,DeliveryMethod,Adm.Cigarettes_daily,1.253698,0.201894,1.109271,1.398125,10,True,0.006605
28,DeliveryMethod,Delivery.Year,1.212968,0.188465,1.078148,1.347788,10,True,0.005983
181,DeliveryMethod,County_anderson,0.581823,0.158476,0.468456,0.695191,10,True,0.003936
157,DeliveryMethod,County_unicoi,0.364315,0.221114,0.206140,0.522491,9,True,0.001724
53,DeliveryMethod,County_lee,0.242130,0.181411,0.112356,0.371903,9,True,0.001194
50,DeliveryMethod,reason_for_admission_ROM,0.405086,0.232342,0.238879,0.571293,10,True,0.001071
21,DeliveryMethod,laceration_type_vaginal,0.645743,0.199327,0.503153,0.788334,10,True,0.000916


### data:ml

In [40]:
X_train, X_test, y_train, y_test = helpers.split_and_scale_data(features_ml, labels_ml)

In [41]:
y_train.value_counts()

0    8631
1    3703
Name: C/S, dtype: int64

In [42]:
y_test.value_counts()

0    3691
1    1595
Name: C/S, dtype: int64

In [43]:
X_train, y_train = helpers.perform_SMOTE(X_train, y_train)
y_train.value_counts()

1    8631
0    8631
Name: C/S, dtype: int64

In [44]:
LASSO_train, LASSO_test, output, model = models.run_LASSO2(X_train, X_test, y_train, y_test, study='DeliveryMethod')

/Users/menglinghe/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [45]:
y_pred = model.predict(X_test)
y_pred_binary = [0 if elem <= 0.5 else 1 for elem in y_pred]

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred_binary))
print('Precision:', precision_score(y_test, y_pred_binary))
print('Recall:', recall_score(y_test, y_pred_binary))

AUC: 0.996622726296023
Accuracy: 0.9941354521377223
Precision: 0.9899749373433584
Recall: 0.9905956112852664


In [46]:
pd.set_option('display.max_rows', 350)
output[(output['significant'] == True) & (output['coef'] < 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
136,DeliveryMethod,Csection.Incidence_N_A,-3.128933,0.803596,-3.703792,-2.554075,10,True,0.246105
46,DeliveryMethod,Gestational.Status.A_2,-0.606250,0.279064,-0.805880,-0.406619,10,True,0.001485
161,DeliveryMethod,County_hawkins,-0.331596,0.158913,-0.445275,-0.217917,10,True,0.001464
122,DeliveryMethod,Delivery.Month_Sept,-0.383846,0.316016,-0.609910,-0.157782,7,True,0.001362
35,DeliveryMethod,delivery_anesthesia_spinal,-0.612475,0.135774,-0.709602,-0.515348,10,True,0.001152
119,DeliveryMethod,Adm.Illicit.Drugs_YES,-0.329240,0.220223,-0.486778,-0.171701,9,True,0.000718
22,DeliveryMethod,Oxytocin_Aug,-0.233272,0.053417,-0.271484,-0.195060,10,True,0.000551
19,DeliveryMethod,infant_complications_A_acidosis,-0.248308,0.079908,-0.305471,-0.191146,10,True,0.000512
68,DeliveryMethod,County_sullivan,-0.284810,0.136288,-0.382305,-0.187316,9,True,0.000488
30,DeliveryMethod,delivery_anesthesia_intrathe,-0.206157,0.161384,-0.321604,-0.090710,9,True,0.000479


In [47]:
output[(output['significant'] == True) & (output['coef'] > 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
137,DeliveryMethod,reason_for_admission_Spon_Ab,1.760315,0.560585,1.359296,2.161334,10,True,0.057558
138,DeliveryMethod,Delivery.Month_Oct,1.373622,0.711233,0.864837,1.882408,9,True,0.017832
17,DeliveryMethod,Adm.Alcohol_YES,1.279061,0.277594,1.080482,1.477640,10,True,0.009437
28,DeliveryMethod,Delivery.Year,1.242030,0.183633,1.110667,1.373393,10,True,0.007527
184,DeliveryMethod,Attempt.to.Vbac_No,0.622024,0.135284,0.525248,0.718801,10,True,0.005411
121,DeliveryMethod,delivery_anesthesia_cse,0.894965,0.390206,0.615828,1.174101,10,True,0.003304
118,DeliveryMethod,reason_for_admission_PTL,0.263556,0.120788,0.177149,0.349962,10,True,0.002086
21,DeliveryMethod,laceration_type_vaginal,0.789694,0.262050,0.602235,0.977154,10,True,0.001996
160,DeliveryMethod,race_black,0.397680,0.249316,0.219330,0.576030,9,True,0.001721
11,DeliveryMethod,reason_for_admission_Induce,0.590544,0.124301,0.501624,0.679464,10,True,0.001185


### data: mice

In [48]:
X_train, X_test, y_train, y_test = helpers.split_and_scale_data(features_mice, labels_mice)

In [49]:
y_train.value_counts()

0    8631
1    3703
Name: C/S, dtype: int64

In [50]:
y_test.value_counts()

0    3691
1    1595
Name: C/S, dtype: int64

In [51]:
X_train, y_train = helpers.perform_SMOTE(X_train, y_train)
y_train.value_counts()

1    8631
0    8631
Name: C/S, dtype: int64

In [52]:
LASSO_train, LASSO_test, output, model = models.run_LASSO2(X_train, X_test, y_train, y_test, study='DeliveryMethod_mice')

/Users/menglinghe/anaconda3/lib/python3.10/site-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2351: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/Users/menglinghe/anaconda3/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2352: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


In [53]:
y_pred = model.predict(X_test)
y_pred_binary = [0 if elem <= 0.5 else 1 for elem in y_pred]

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

print('AUC:', roc_auc_score(y_test, y_pred))
print('Accuracy:', accuracy_score(y_test, y_pred_binary))
print('Precision:', precision_score(y_test, y_pred_binary))
print('Recall:', recall_score(y_test, y_pred_binary))

AUC: 0.9969402825987809
Accuracy: 0.9945138100643208
Precision: 0.989375
Recall: 0.9924764890282132


In [54]:
pd.set_option('display.max_rows', 350)
output[(output['significant'] == True) & (output['coef'] < 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
87,DeliveryMethod_mice,Provider.On.Admission_Eaton,-1.501539,0.137850,-1.600151,-1.402927,10,True,0.013175
217,DeliveryMethod_mice,County_clay,-0.512320,0.377763,-0.782555,-0.242085,9,True,0.002607
239,DeliveryMethod_mice,race_other_False,-0.780340,0.397319,-1.064565,-0.496115,10,True,0.001987
86,DeliveryMethod_mice,Amniotic.Fluid.Color_clear,-0.691341,0.440951,-1.006778,-0.375903,9,True,0.001353
92,DeliveryMethod_mice,infant_complications_A_tachy_True,-0.393077,0.175613,-0.518703,-0.267452,10,True,0.001233
71,DeliveryMethod_mice,County_carter,-0.538617,0.236158,-0.707555,-0.369680,9,True,0.001218
285,DeliveryMethod_mice,delivery_anesthesia_spinal_False,-0.466935,0.302943,-0.683647,-0.250223,10,True,0.000961
69,DeliveryMethod_mice,Amniotic.Fluid.Color_brown,-0.201600,0.151444,-0.309937,-0.093263,10,True,0.000907
84,DeliveryMethod_mice,County_outofstate,-0.430771,0.188987,-0.565965,-0.295578,10,True,0.000682
254,DeliveryMethod_mice,race_asian_False,-0.190025,0.084100,-0.250186,-0.129863,9,True,0.000661


In [55]:
output[(output['significant'] == True) & (output['coef'] > 0)].sort_values('permutation_importance', ascending=False).head(20)

,study_name,feature_name,coef,coef_sd,lower_95,upper_95,count,significant,permutation_importance
88,DeliveryMethod_mice,infant_complications_A_oligohyd_False,3.456054,0.200972,3.312288,3.599821,10,True,0.483399
89,DeliveryMethod_mice,Ethnicity_D,3.017776,0.395625,2.734762,3.300789,10,True,0.362041
135,DeliveryMethod_mice,Amniotic.Fluid.Color_bloody,0.740811,0.266555,0.550129,0.931493,10,True,0.007575
218,DeliveryMethod_mice,infant_complications_A_brady_False,0.654731,0.320313,0.425592,0.883869,10,True,0.002972
240,DeliveryMethod_mice,delivery_anesthesia_intrathe_True,0.604487,0.313459,0.380252,0.828722,10,True,0.001826
111,DeliveryMethod_mice,reason_for_admission_other_False,0.414166,0.186558,0.280711,0.547621,10,True,0.001658
73,DeliveryMethod_mice,Delivery.Month_Mar,0.494167,0.304810,0.276119,0.712214,9,True,0.000901
290,DeliveryMethod_mice,Episiotomy_None,0.108640,0.116795,0.025090,0.192190,7,True,0.000889
253,DeliveryMethod_mice,Fetal.Presentation.A_Compound,0.254820,0.097388,0.185153,0.324488,10,True,0.000814
11,DeliveryMethod_mice,Provider.On.Admission_ROCK,0.366690,0.089708,0.302517,0.430863,10,True,0.000748


####  complete